In [602]:
import requests
from bs4 import BeautifulSoup
import os
from openai import OpenAI

# 'https://menet.com.cn

In [603]:
def get_list():
    host ='https://menet.com.cn'
    response = requests.post(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = soup.find_all('h3')
    return news_list

In [604]:
host ='https://menet.com.cn'
response = requests.post(url)
soup = BeautifulSoup(response.text, 'html.parser')
news_list = soup.find_all('h3')

In [610]:
news_list = [_ for _ in get_list() if _['class'] == ['item_title']]

In [612]:
def get_info(item):
    host ='https://menet.com.cn'
    link = host+item.a['href']
    title = item.text
    return {
    "title": title,
    'link': link
}

In [613]:
result = list(map(get_info, news_list))

In [662]:
for num, item in enumerate(soup.find_all(lambda tag: tag.has_attr('class') and len(tag)==1 and tag['class'] == ["item_time"])):
    result[num]['date'] = item.text

In [666]:
result

[{'title': '山东新时代爆发了！拿下9个重磅品种，猛攻1100亿市场，11款1类新药、44个新品蓄力',
  'link': 'https://menet.com.cn/info/202409/20240905102904294_148064.shtml',
  'date': '2024-09-05'},
 {'title': '492只医药股上半年成绩单！迈瑞净利润首破70亿，5家晋身10亿军团，甘李、海思科……翻倍',
  'link': 'https://menet.com.cn/info/202409/202409051027242724_148062.shtml',
  'date': '2024-09-05'},
 {'title': '63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏',
  'link': 'https://menet.com.cn/info/202409/202409050941374137_148059.shtml',
  'date': '2024-09-05'},
 {'title': '信立泰厉害了！12款1类新药曝光，猛攻1300亿市场，5款新药上市可期',
  'link': 'https://menet.com.cn/info/202409/20240903141804184_148053.shtml',
  'date': '2024-09-03'},
 {'title': '“中药股王”发威！肝病用药狂卖26亿，14个独家中成药热销，8款重磅新药出击',
  'link': 'https://menet.com.cn/info/202409/202409031416261626_148051.shtml',
  'date': '2024-09-03'},
 {'title': '第十批集采24个品种备战！千亿市场齐鲁亮了，2大注射剂疯狂连涨，倍特、华海蓄力',
  'link': 'https://menet.com.cn/info/202409/20240903141404144_148049.shtml',
  'date': '2024-09-03'},
 {'title': '28款儿科中成药亮眼！葵花、健民、济川领跑实体药店，3大品牌网上药店翻倍',


# https://www.pharnexcloud.com/zixun

In [976]:
r = requests.get('https://www.pharnexcloud.com/zixun')

In [977]:
soup = BeautifulSoup(r.text, 'html.parser')

In [978]:
dates = soup.find_all(lambda tag:tag.has_attr('class') and tag['class']==['date'])

In [1002]:
dates = [_.text for _ in dates]

In [986]:
result = soup.find_all('a', class_ = 'img-content')

In [992]:
def query_tilte_url(item):
    title = item.find_next()['alt']
    url = 'https://www.pharnexcloud.com' + item['href']
    return {
        'title': title,
        'url': url
    }

In [1004]:
res = list(map(query_tilte_url, result))

In [1016]:
for no, item in enumerate(res):
    
    item['date'] = dates[no]

0 {'title': '齐鲁制药再添利器！美罗培南普莱巴坦获临床试验许可，专攻复杂性腹腔感染', 'url': 'https://www.pharnexcloud.com/zixun/sx_8459'}
1 {'title': '科伦博泰SKB571双抗ADC药物IND申请获NMPA受理，肿瘤治疗迎新突破', 'url': 'https://www.pharnexcloud.com/zixun/sx_8458'}
2 {'title': '华润九新：头孢唑林钠水合物日本成功上市！质量卓越引领市场', 'url': 'https://www.pharnexcloud.com/zixun/sx_8457'}
3 {'title': '百奥泰度普利尤单抗BAT2406获NMPA临床试验批准，助力特应性皮炎等疾病治疗', 'url': 'https://www.pharnexcloud.com/zixun/sx_8456'}
4 {'title': '康方生物依沃西：全球首创PD-1/VEGF双抗，III期临床力压帕博利珠单抗！', 'url': 'https://www.pharnexcloud.com/zixun/sx_8455'}
5 {'title': '恒瑞医药SHR-1918注射液拟获突破性治疗认定，降脂新利器亮相ESC！', 'url': 'https://www.pharnexcloud.com/zixun/sx_8454'}
6 {'title': 'AI赋能药物革命：礼来与Genetic Leap合作开发RNA靶向疗法', 'url': 'https://www.pharnexcloud.com/zixun/sx_8453'}
7 {'title': '全球监管合作加速基因疗法普及：FDA专家展望五年蓝图', 'url': 'https://www.pharnexcloud.com/zixun/sx_8452'}
8 {'title': '医疗耗材集采新动向：深圳15+12省联盟聚焦超声刀头与预充式导管冲洗器', 'url': 'https://www.pharnexcloud.com/zixun/sx_8451'}
9 {'title': '人福医药创新药物HW071021片获批临床，抗肿瘤药市场再添利器', 'url': 'https://www.

In [1022]:
res

[{'title': '齐鲁制药再添利器！美罗培南普莱巴坦获临床试验许可，专攻复杂性腹腔感染',
  'url': 'https://www.pharnexcloud.com/zixun/sx_8459',
  'date': '2024-09-09 11:56:32'},
 {'title': '科伦博泰SKB571双抗ADC药物IND申请获NMPA受理，肿瘤治疗迎新突破',
  'url': 'https://www.pharnexcloud.com/zixun/sx_8458',
  'date': '2024-09-09 11:46:09'},
 {'title': '华润九新：头孢唑林钠水合物日本成功上市！质量卓越引领市场',
  'url': 'https://www.pharnexcloud.com/zixun/sx_8457',
  'date': '2024-09-09 11:28:58'},
 {'title': '百奥泰度普利尤单抗BAT2406获NMPA临床试验批准，助力特应性皮炎等疾病治疗',
  'url': 'https://www.pharnexcloud.com/zixun/sx_8456',
  'date': '2024-09-09 11:16:12'},
 {'title': '康方生物依沃西：全球首创PD-1/VEGF双抗，III期临床力压帕博利珠单抗！',
  'url': 'https://www.pharnexcloud.com/zixun/sx_8455',
  'date': '2024-09-09 10:59:31'},
 {'title': '恒瑞医药SHR-1918注射液拟获突破性治疗认定，降脂新利器亮相ESC！',
  'url': 'https://www.pharnexcloud.com/zixun/sx_8454',
  'date': '2024-09-09 10:51:50'},
 {'title': 'AI赋能药物革命：礼来与Genetic Leap合作开发RNA靶向疗法',
  'url': 'https://www.pharnexcloud.com/zixun/sx_8453',
  'date': '2024-09-09 10:41:53'},
 {'title': '全球监管合作加速基因

---

# https://bydrug.pharmcube.com/

In [1026]:
r = requests.get('https://bydrug.pharmcube.com/')

In [1028]:
soup = BeautifulSoup(r.text, 'html.parser')

In [1030]:
soup

<!DOCTYPE html>

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22zh%22%7D%7D" data-n-head-ssr="" lang="zh">
<head>
<meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1.0, minimum-scale=1, maximum-scale=1, user-scalable=no" data-n-head="ssr" name="viewport"/><meta content="telephone=no" data-n-head="ssr" name="format-detection"/><meta content="codeva-eL5dxvvOkL" data-n-head="ssr" name="baidu-site-verification"/><meta content="strict-origin-when-cross-origin" data-n-head="ssr" name="referrer"/><meta content="研究报告，医药魔方独家研究报告，海外研究报告，药企演讲ppt，药企财报，招股书，招股说明书，CDE培训咨询会议，FDA咨询会议，医药行业会议，药品获批新闻，医药新闻，医药魔方深度分析，热点研究，中国医疗器械，医疗器械上市数据，医疗器械注册数据，医疗器械注册审批信息，获批新药，license out，国家集采中标，医保谈判结果，ByDrug，掰药" data-n-head="ssr" name="keywords"/><meta content="ByDrug掰药是一个专业的、覆盖医药行业药械相关信息资源和媒体资讯服务的共享中心。聚焦于药物和器械研发信息，面向医药行业从业人员提供医药行业优质研究报告、新闻报道、医药类专利和中国及海外医疗器械数据等。" data-hid="description" data-n-head="ssr" name="description"/><title>ByDrug-一站式医药资源共享中心-医药魔方</title><link data-n-head="s

In [1131]:
datas = soup.find_all('div', class_= 'mf-flex mf-items-center')

In [1133]:
datas = datas[1:11]

In [1135]:
def get_tilte_and_url(item):
    title = item.text.strip()
    url = item.a['href']
    return {
        'title': title,
        'url': url
    }

In [1137]:
datas[0]

<div class="mf-flex mf-items-center" data-v-23503f46=""><a class="fileName" data-v-23503f46="" href="https://bydrug.pharmcube.com/report/detail/bbd1d34c596d4ece9403a1b1e3c3ff0a" style="max-width:1080px;" target="_blank">
          医药行业周报：武田1类创新药拟纳入突破性治疗品种
        </a> <!-- --></div>

In [1139]:
get_tilte_and_url(datas[0])

{'title': '医药行业周报：武田1类创新药拟纳入突破性治疗品种',
 'url': 'https://bydrug.pharmcube.com/report/detail/bbd1d34c596d4ece9403a1b1e3c3ff0a'}

In [1143]:
res = list(map(get_tilte_and_url, datas))

In [1145]:
dates = [_.text for _ in soup.find_all('span', class_= 'text') if '-' in _.text]

In [1153]:
dates

['2024-09-09',
 '2024-09-09',
 '2024-09-09',
 '2024-09-09',
 '2024-09-08',
 '2024-09-08',
 '2024-09-08',
 '2024-09-08',
 '2024-09-08',
 '2024-09-08']

In [1157]:
for no, _ in enumerate(res):
    _['date'] = dates[no]

# https://news.yaozh.com/archivelist/24

In [1162]:
r = requests.get('https://bydrug.pharmcube.com/')

In [1164]:
soup = BeautifulSoup(r.text, 'html.parser')

In [1166]:
soup

<!DOCTYPE html>

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22zh%22%7D%7D" data-n-head-ssr="" lang="zh">
<head>
<meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1.0, minimum-scale=1, maximum-scale=1, user-scalable=no" data-n-head="ssr" name="viewport"/><meta content="telephone=no" data-n-head="ssr" name="format-detection"/><meta content="codeva-eL5dxvvOkL" data-n-head="ssr" name="baidu-site-verification"/><meta content="strict-origin-when-cross-origin" data-n-head="ssr" name="referrer"/><meta content="研究报告，医药魔方独家研究报告，海外研究报告，药企演讲ppt，药企财报，招股书，招股说明书，CDE培训咨询会议，FDA咨询会议，医药行业会议，药品获批新闻，医药新闻，医药魔方深度分析，热点研究，中国医疗器械，医疗器械上市数据，医疗器械注册数据，医疗器械注册审批信息，获批新药，license out，国家集采中标，医保谈判结果，ByDrug，掰药" data-n-head="ssr" name="keywords"/><meta content="ByDrug掰药是一个专业的、覆盖医药行业药械相关信息资源和媒体资讯服务的共享中心。聚焦于药物和器械研发信息，面向医药行业从业人员提供医药行业优质研究报告、新闻报道、医药类专利和中国及海外医疗器械数据等。" data-hid="description" data-n-head="ssr" name="description"/><title>ByDrug-一站式医药资源共享中心-医药魔方</title><link data-n-head="s